In [12]:
import pandas as pd
#from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder

import random
import numpy as np
from tqdm import tqdm
#from tensorflow.python.keras.preprocessing.sequence import pad_sequences
#from deepmatch.models import *
#from deepmatch.utils import sampledsoftmaxloss


import warnings
warnings.filterwarnings("ignore")

In [29]:
x = [1,3,4,4,5,5]
print(x[::-1])

[5, 5, 4, 4, 3, 1]


In [13]:
path='./ml-1m/'
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
mnames = ['movie_id', 'title', 'genres']

users = pd.read_csv(path+'users.dat', sep='::', header=None, names=unames,engine='python',encoding='ISO-8859-1')
ratings = pd.read_table(path+'ratings.dat', sep='::', header=None, names=rnames, engine='python')
movies = pd.read_table(path+'movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='ISO-8859-1')
#encoding='ISO-8859-1'
#解决‘utf-8‘ codec can‘t decode byte 0xe9 in position 3114: invalid continuation byte 错误

data = pd.merge(pd.merge(ratings, users, on='user_id'), movies, on='movie_id').sample(2000)
data.info



<bound method DataFrame.info of         user_id  movie_id  rating   timestamp gender  age  occupation    zip  \
845938     5306      2250       4   961016927      F   25          17  92121   
18049      4302      2791       3   965585416      M   50           1  45385   
467005     1748      1294       4   974710043      M   50           1  04240   
743978     2424      1388       1   974691028      M   25          17  95035   
798506     1425       144       3  1038547457      M    1          10  08872   
...         ...       ...     ...         ...    ...  ...         ...    ...   
595487     3312      2719       2   990954806      F   18           4  90039   
255404     5359       377       5   960556258      M   18          16  27705   
460050     5492       329       1   961957545      M   18           4  55104   
754169     1194      3361       3  1036310539      F   25           1  95617   
777698     2114      2403       4   997548322      M   18          18  21740   

       

In [14]:
data.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
845938,5306,2250,4,961016927,F,25,17,92121,Men Don't Leave (1990),Drama
18049,4302,2791,3,965585416,M,50,1,45385,Airplane! (1980),Comedy
467005,1748,1294,4,974710043,M,50,1,04240,M*A*S*H (1970),Comedy|War
743978,2424,1388,1,974691028,M,25,17,95035,Jaws 2 (1978),Action|Horror
798506,1425,144,3,1038547457,M,1,10,08872,"Brothers McMullen, The (1995)",Comedy


In [15]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", ]
SEQ_LEN = 50
negsample = 3

In [16]:
features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']

feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1  # 默认编码从０开始
    feature_max_idx[feature] = data[feature].max() + 1  # 作为Embedding向量的大小设定

In [17]:
# 构建用户画像
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
user_profile


,gender,age,occupation,zip
user_id,,,,
1281,1,3,18,953
1045,2,6,2,458
446,2,6,2,69
611,2,3,18,1047
353,2,1,11,123
...,...,...,...,...
97,2,3,12,34
1009,2,2,20,829
309,2,4,3,233


In [18]:
# 构建物品画像
item_profile = data[["movie_id"]].drop_duplicates('movie_id')
item_profile.head()

,movie_id
845938,678
18049,827
467005,414
743978,449
798506,38


In [45]:
# 用户历史点击文章序列
user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list

user_id
1       [800, 898]
2            [503]
3       [1104, 28]
4       [291, 405]
5            [648]
           ...    
1464    [660, 176]
1465        [1065]
1466         [556]
1467         [615]
1468    [451, 417]
Name: movie_id, Length: 1468, dtype: object

In [21]:
data.sort_values("timestamp", inplace=True)
item_ids = data['movie_id'].unique()
negsample=3



100%|████████████████████████████████████████████████████████████████████████████| 1468/1468 [00:00<00:00, 2648.11it/s]

6 6


In [22]:
item_ids

array([451, 615, 417, ..., 906,  59, 745], dtype=int64)

In [23]:
data

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
231374,1468,451,3,956712084,1,3,16,360,Citizen Ruth (1996),Comedy|Drama
424189,1467,615,3,956726304,2,4,7,155,"Honey, I Shrunk the Kids (1989)",Adventure|Children's|Comedy|Fantasy|Sci-Fi
771613,1468,417,4,956753647,1,3,16,360,Pink Floyd - The Wall (1982),Drama|Musical|War
247178,1466,556,4,956757546,2,3,13,124,"X-Files: Fight the Future, The (1998)",Mystery|Sci-Fi|Thriller
32686,1463,521,4,956839540,2,5,18,337,Titanic (1997),Drama|Romance
...,...,...,...,...,...,...,...,...,...,...
279620,1427,816,5,1043784745,1,3,1,666,"Mosquito Coast, The (1986)",Drama
621352,103,557,3,1044901116,2,2,4,604,Doctor Dolittle (1998),Comedy
551084,1204,59,5,1044998167,2,2,8,594,"To Wong Foo, Thanks for Everything! Julie Newm...",Comedy
481197,823,1026,2,1046188847,2,2,5,492,Hook (1991),Adventure|Fantasy


In [46]:
group = data.groupby('user_id')
for reviewerID, hist in data.groupby('user_id'):
    print(reviewerID)
    print(hist)

1
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
239642        1       800       4  978243085       2    3          21  613   
275217        1       898       1  978245916       2    3          21  613   

                                              title            genres  
239642  South Park: Bigger, Longer and Uncut (1999)  Animation|Comedy  
275217                          Falling Down (1993)      Action|Drama  
2
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
369183        2       503       4  978226109       2    3          18  672   

                     title         genres  
369183  Chasing Amy (1997)  Drama|Romance  
3
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
407458        3      1104       3  978219385       1    3           2   66   
570458        3        28       4  978904663       1    3           2   66   

                            title                     genres  


        user_id  movie_id  rating   timestamp  gender  age  occupation  zip  \
270508       97       562       5  1020427185       2    3          12   34   

            title           genres  
270508  Pi (1998)  Sci-Fi|Thriller  
98
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
446772       98       405       3  976289117       2    2          13  579   

                            title         genres  
446772  Young Frankenstein (1974)  Comedy|Horror  
99
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
822624       99      1031       3  978451779       1    5           2  424   

                          title genres  
822624  Midnight Express (1978)  Drama  
100
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
392927      100        64       4  976553818       2    4          16  577   

                title  genres  
392927  Clerks (1994)  Comedy  
101
        user_id  movie_id  rating 

        user_id  movie_id  rating  timestamp  gender  age  occupation   zip  \
375550      203        34       5  977182014       1    3           1  1087   

                     title          genres  
375550  Taxi Driver (1976)  Drama|Thriller  
204
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
974431      204       567       4  975425631       2    7           7  225   

                                     title genres  
974431  Greatest Show on Earth, The (1952)  Drama  
205
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
355036      205       452       4  975272374       2    3           1  905   

                       title         genres  
355036  Jerry Maguire (1996)  Drama|Romance  
206
       user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
35178      206       187       5  998427568       2    1          11  512   

                title                               genres  
35178  Aladdi

       user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
49925      308       294       5  974815705       1    5           7  553   

                     title                     genres  
49925  Mary Poppins (1964)  Children's|Comedy|Musical  
309
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
184880      309       453       4  974815156       2    4           3  233   

                         title  genres  
184880  Raising Arizona (1987)  Comedy  
310
      user_id  movie_id  rating  timestamp  gender  age  occupation   zip  \
2399      310       255       3  974856611       2    4           2  1143   

                    title           genres  
2399  My Fair Lady (1964)  Musical|Romance  
311
        user_id  movie_id  rating  timestamp  gender  age  occupation   zip  \
107776      311       846       4  974792948       2    3          20  1089   

                         title        genres  
107776  American Beauty (1999) 

        user_id  movie_id  rating  timestamp  gender  age  occupation   zip  \
338031      425       283       3  974707383       2    3          13  1124   

                                  title                     genres  
338031  Apple Dumpling Gang, The (1975)  Children's|Comedy|Western  
426
        user_id  movie_id  rating  timestamp  gender  age  occupation   zip  \
530175      426       365       4  974707652       2    3           2  1088   

                          title                                 genres  
530175  Army of Darkness (1993)  Action|Adventure|Comedy|Horror|Sci-Fi  
427
        user_id  movie_id  rating  timestamp  gender  age  occupation   zip  \
657400      427       650       4  974707381       1    2           5  1115   

                                             title  \
657400  American Tail: Fievel Goes West, An (1991)   

                             genres  
657400  Animation|Children's|Comedy  
428
        user_id  movie_id  rating  timesta

...

        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
881996     1034       114       4  965309053       1    6           1  377   

                        title         genres  
881996  I Love Trouble (1994)  Action|Comedy  
1035
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
793614     1035        60       3  965306895       2    6          17  403   

                               title         genres  
793614  Walk in the Clouds, A (1995)  Drama|Romance  
1036
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
467493     1036       414       3  965305703       2    3          17  625   
119661     1036       618       3  965305732       2    3          17  625   

                        title             genres  
467493         M*A*S*H (1970)         Comedy|War  
119661  Doctor Zhivago (1965)  Drama|Romance|War  
1037
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
502

        user_id  movie_id  rating  timestamp  gender  age  occupation   zip  \
619015     1147       321       3  963331798       2    3           4  1006   

                        title            genres  
619015  Basic Instinct (1992)  Mystery|Thriller  
1148
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
573442     1148       990       2  963249450       1    2           5   82   

                           title              genres  
573442  Dog Day Afternoon (1975)  Comedy|Crime|Drama  
1149
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
905932     1149       710       4  963241926       2    2           5  303   

                           title         genres  
905932  Godzilla (Gojira) (1984)  Action|Sci-Fi  
1150
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
559776     1150        69       5  963190725       1    3           1  918   
930497     1150       200       3  963194185  

        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
94262      1235       967       3  962238810       2    3           8  930   
740160     1235         6       5  962238930       2    3           8  930   

                                 title                genres  
94262    League of Their Own, A (1992)          Comedy|Drama  
740160  American President, The (1995)  Comedy|Drama|Romance  
1236
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
968894     1236       689       3  962200939       2    4           3  258   

                        title  genres  
968894  Impostors, The (1998)  Comedy  
1237
        user_id  movie_id  rating   timestamp  gender  age  occupation  zip  \
483679     1237       259       5  1036269921       2    4           8  783   

                      title genres  
483679  Citizen Kane (1941)  Drama  
1238
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
430598     12

        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
685299     1338       322       4  959815821       2    3          17  710   
901595     1338       198       1  959992283       2    3          17  710   

                               title                genres  
685299       Crying Game, The (1992)     Drama|Romance|War  
901595  Hellraiser: Bloodline (1996)  Action|Horror|Sci-Fi  
1339
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
595652     1339       811       1  959498756       2    3          16  124   

                       title           genres  
595652  Haunting, The (1999)  Horror|Thriller  
1340
       user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
32573     1340       521       4  959477483       1    2           5  141   

                title         genres  
32573  Titanic (1997)  Drama|Romance  
1341
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
110

        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
700461     1437       589       4  957362931       2    3           2  777   

                     title           genres  
700461  Poltergeist (1982)  Horror|Thriller  
1438
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
668991     1438       679       3  957403520       2    2           5  624   

              title                 genres  
668991  Toys (1992)  Action|Comedy|Fantasy  
1439
        user_id  movie_id  rating  timestamp  gender  age  occupation  zip  \
648079     1439       419       4  957321271       2    6          21  499   
128316     1439       189       3  957321396       2    6          21  499   
820015     1439       694       4  957359929       2    6          21  499   

                            title                   genres  
648079    Forbidden Planet (1956)                   Sci-Fi  
128316  Dances with Wolves (1990)  Adventure|Drama|Western 

In [41]:
def gen_data_set(data, negsample=0):

    data.sort_values("timestamp", inplace=True)
    item_ids = data['movie_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['movie_id'].tolist()
        rating_list = hist['rating'].tolist()

        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1]),rating_list[i]))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
            else:
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),rating_list[i]))

    #random.shuffle(train_set)
    #random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))

    return train_set,test_set

In [42]:
train_set, test_set = gen_data_set(data, negsample)

100%|████████████████████████████████████████████████████████████████████████████| 1468/1468 [00:00<00:00, 2807.85it/s]

6 6


In [43]:
train_set

[(15, [523], 443, 1, 1, 3),
 (15, [523], 421, 0, 1),
 (15, [523], 285, 0, 1),
 (15, [523], 530, 0, 1),
 (15, [443, 523], 349, 1, 2, 4),
 (15, [443, 523], 985, 0, 2),
 (15, [443, 523], 994, 0, 2),
 (15, [443, 523], 476, 0, 2),
 (44, [808], 877, 1, 1, 4),
 (44, [808], 118, 0, 1),
 (44, [808], 38, 0, 1),
 (44, [808], 635, 0, 1),
 (46, [908], 543, 1, 1, 4),
 (46, [908], 193, 0, 1),
 (46, [908], 450, 0, 1),
 (46, [908], 382, 0, 1),
 (64, [1060], 1043, 1, 1, 5),
 (64, [1060], 354, 0, 1),
 (64, [1060], 1132, 0, 1),
 (64, [1060], 376, 0, 1),
 (105, [377], 680, 1, 1, 2),
 (105, [377], 1071, 0, 1),
 (105, [377], 458, 0, 1),
 (105, [377], 1037, 0, 1),
 (105, [680, 377], 204, 1, 2, 4),
 (105, [680, 377], 409, 0, 2),
 (105, [680, 377], 911, 0, 2),
 (105, [680, 377], 374, 0, 2),
 (105, [204, 680, 377], 241, 1, 3, 3),
 (105, [204, 680, 377], 166, 0, 3),
 (105, [204, 680, 377], 508, 0, 3),
 (105, [204, 680, 377], 1104, 0, 3),
 (112, [215], 809, 1, 1, 2),
 (112, [215], 211, 0, 1),
 (112, [215], 1122, 0

In [44]:
test_set

[(1, [800], 898, 1, 1, 1),
 (3, [1104], 28, 1, 1, 4),
 (4, [291], 405, 1, 1, 5),
 (8, [619], 770, 1, 1, 4),
 (13, [1057], 499, 1, 1, 5),
 (15, [349, 443, 523], 622, 1, 3, 4),
 (24, [400], 751, 1, 1, 2),
 (30, [232], 656, 1, 1, 3),
 (31, [116], 669, 1, 1, 3),
 (32, [133], 358, 1, 1, 5),
 (36, [249], 273, 1, 1, 3),
 (44, [877, 808], 369, 1, 2, 4),
 (46, [543, 908], 542, 1, 2, 5),
 (48, [875], 99, 1, 1, 3),
 (55, [256], 435, 1, 1, 5),
 (64, [1043, 1060], 368, 1, 2, 4),
 (70, [191], 518, 1, 1, 4),
 (71, [76], 530, 1, 1, 1),
 (72, [416], 782, 1, 1, 4),
 (77, [836], 426, 1, 1, 5),
 (79, [473], 56, 1, 1, 2),
 (82, [109], 21, 1, 1, 5),
 (89, [806], 816, 1, 1, 5),
 (93, [642], 975, 1, 1, 2),
 (94, [343], 941, 1, 1, 5),
 (105, [241, 204, 680, 377], 762, 1, 4, 3),
 (112, [809, 215], 734, 1, 2, 2),
 (114, [508], 10, 1, 1, 4),
 (115, [361], 1099, 1, 1, 4),
 (116, [1083, 455], 145, 1, 2, 3),
 (121, [1050], 231, 1, 1, 4),
 (124, [632], 661, 1, 1, 3),
 (127, [1073], 78, 1, 1, 2),
 (128, [362], 969, 1,

In [30]:
train_set = []
test_set = []
for reviewerID, hist in tqdm(data.groupby('user_id')):
    pos_list = hist['movie_id'].tolist()
    rating_list = hist['rating'].tolist()

    if negsample > 0:
        candidate_set = list(set(item_ids) - set(pos_list))
        neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
    for i in range(1, len(pos_list)):
        hist = pos_list[:i]
        if i != len(pos_list) - 1:
            train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1]),rating_list[i]))
            print(train_set)
            for negi in range(negsample):
                train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
        else:
            test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),rating_list[i]))

random.shuffle(train_set)
random.shuffle(test_set)

print(len(train_set[0]),len(test_set[0]))

 15%|███████████▋                                                                 | 222/1468 [00:00<00:00, 2204.37it/s]

[(15, [523], 443, 1, 1, 3)]
[(15, [523], 443, 1, 1, 3), (15, [523], 996, 0, 1), (15, [523], 57, 0, 1), (15, [523], 884, 0, 1), (15, [443, 523], 349, 1, 2, 4)]
[(15, [523], 443, 1, 1, 3), (15, [523], 996, 0, 1), (15, [523], 57, 0, 1), (15, [523], 884, 0, 1), (15, [443, 523], 349, 1, 2, 4), (15, [443, 523], 901, 0, 2), (15, [443, 523], 152, 0, 2), (15, [443, 523], 633, 0, 2), (44, [808], 877, 1, 1, 4)]
[(15, [523], 443, 1, 1, 3), (15, [523], 996, 0, 1), (15, [523], 57, 0, 1), (15, [523], 884, 0, 1), (15, [443, 523], 349, 1, 2, 4), (15, [443, 523], 901, 0, 2), (15, [443, 523], 152, 0, 2), (15, [443, 523], 633, 0, 2), (44, [808], 877, 1, 1, 4), (44, [808], 988, 0, 1), (44, [808], 489, 0, 1), (44, [808], 231, 0, 1), (46, [908], 543, 1, 1, 4)]
[(15, [523], 443, 1, 1, 3), (15, [523], 996, 0, 1), (15, [523], 57, 0, 1), (15, [523], 884, 0, 1), (15, [443, 523], 349, 1, 2, 4), (15, [443, 523], 901, 0, 2), (15, [443, 523], 152, 0, 2), (15, [443, 523], 633, 0, 2), (44, [808], 877, 1, 1, 4), (44, [8

 59%|█████████████████████████████████████████████▎                               | 864/1468 [00:00<00:00, 2026.89it/s]

[(15, [523], 443, 1, 1, 3), (15, [523], 996, 0, 1), (15, [523], 57, 0, 1), (15, [523], 884, 0, 1), (15, [443, 523], 349, 1, 2, 4), (15, [443, 523], 901, 0, 2), (15, [443, 523], 152, 0, 2), (15, [443, 523], 633, 0, 2), (44, [808], 877, 1, 1, 4), (44, [808], 988, 0, 1), (44, [808], 489, 0, 1), (44, [808], 231, 0, 1), (46, [908], 543, 1, 1, 4), (46, [908], 844, 0, 1), (46, [908], 561, 0, 1), (46, [908], 869, 0, 1), (64, [1060], 1043, 1, 1, 5), (64, [1060], 55, 0, 1), (64, [1060], 8, 0, 1), (64, [1060], 735, 0, 1), (105, [377], 680, 1, 1, 2), (105, [377], 36, 0, 1), (105, [377], 1088, 0, 1), (105, [377], 407, 0, 1), (105, [680, 377], 204, 1, 2, 4), (105, [680, 377], 481, 0, 2), (105, [680, 377], 373, 0, 2), (105, [680, 377], 711, 0, 2), (105, [204, 680, 377], 241, 1, 3, 3), (105, [204, 680, 377], 783, 0, 3), (105, [204, 680, 377], 624, 0, 3), (105, [204, 680, 377], 13, 0, 3), (112, [215], 809, 1, 1, 2), (112, [215], 462, 0, 1), (112, [215], 267, 0, 1), (112, [215], 858, 0, 1), (116, [455],

 73%|███████████████████████████████████████████████████████▏                    | 1067/1468 [00:00<00:00, 1939.51it/s]

[(15, [523], 443, 1, 1, 3), (15, [523], 996, 0, 1), (15, [523], 57, 0, 1), (15, [523], 884, 0, 1), (15, [443, 523], 349, 1, 2, 4), (15, [443, 523], 901, 0, 2), (15, [443, 523], 152, 0, 2), (15, [443, 523], 633, 0, 2), (44, [808], 877, 1, 1, 4), (44, [808], 988, 0, 1), (44, [808], 489, 0, 1), (44, [808], 231, 0, 1), (46, [908], 543, 1, 1, 4), (46, [908], 844, 0, 1), (46, [908], 561, 0, 1), (46, [908], 869, 0, 1), (64, [1060], 1043, 1, 1, 5), (64, [1060], 55, 0, 1), (64, [1060], 8, 0, 1), (64, [1060], 735, 0, 1), (105, [377], 680, 1, 1, 2), (105, [377], 36, 0, 1), (105, [377], 1088, 0, 1), (105, [377], 407, 0, 1), (105, [680, 377], 204, 1, 2, 4), (105, [680, 377], 481, 0, 2), (105, [680, 377], 373, 0, 2), (105, [680, 377], 711, 0, 2), (105, [204, 680, 377], 241, 1, 3, 3), (105, [204, 680, 377], 783, 0, 3), (105, [204, 680, 377], 624, 0, 3), (105, [204, 680, 377], 13, 0, 3), (112, [215], 809, 1, 1, 2), (112, [215], 462, 0, 1), (112, [215], 267, 0, 1), (112, [215], 858, 0, 1), (116, [455],

100%|████████████████████████████████████████████████████████████████████████████| 1468/1468 [00:00<00:00, 1978.27it/s]

[(15, [523], 443, 1, 1, 3), (15, [523], 996, 0, 1), (15, [523], 57, 0, 1), (15, [523], 884, 0, 1), (15, [443, 523], 349, 1, 2, 4), (15, [443, 523], 901, 0, 2), (15, [443, 523], 152, 0, 2), (15, [443, 523], 633, 0, 2), (44, [808], 877, 1, 1, 4), (44, [808], 988, 0, 1), (44, [808], 489, 0, 1), (44, [808], 231, 0, 1), (46, [908], 543, 1, 1, 4), (46, [908], 844, 0, 1), (46, [908], 561, 0, 1), (46, [908], 869, 0, 1), (64, [1060], 1043, 1, 1, 5), (64, [1060], 55, 0, 1), (64, [1060], 8, 0, 1), (64, [1060], 735, 0, 1), (105, [377], 680, 1, 1, 2), (105, [377], 36, 0, 1), (105, [377], 1088, 0, 1), (105, [377], 407, 0, 1), (105, [680, 377], 204, 1, 2, 4), (105, [680, 377], 481, 0, 2), (105, [680, 377], 373, 0, 2), (105, [680, 377], 711, 0, 2), (105, [204, 680, 377], 241, 1, 3, 3), (105, [204, 680, 377], 783, 0, 3), (105, [204, 680, 377], 624, 0, 3), (105, [204, 680, 377], 13, 0, 3), (112, [215], 809, 1, 1, 2), (112, [215], 462, 0, 1), (112, [215], 267, 0, 1), (112, [215], 858, 0, 1), (116, [455],

In [39]:
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])
    
    # 填补缺失值
    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    
    # 标签　movie_id:最后一次的点击视频id号
    train_model_input = {"user_id": train_uid, "movie_id": train_iid, "hist_movie_id": train_seq_pad,
                         "hist_len": train_hist_len}
    
    # 添加用户信息
    for key in ["gender", "age", "occupation", "zip"]:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input, train_label

In [ ]:
train_set, test_set = gen_data_set(data, negsample)
#train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
#test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

  7%|█████▋                                                                         | 434/6040 [00:03<01:10, 79.70it/s]

In [ ]:
embedding_dim = 16
user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
                        SparseFeat("gender", feature_max_idx['gender'], embedding_dim),
                        SparseFeat("age", feature_max_idx['age'], embedding_dim),
                        SparseFeat("occupation", feature_max_idx['occupation'], embedding_dim),
                        SparseFeat("zip", feature_max_idx['zip'], embedding_dim),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]


item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]